In [2]:
from sklearn.base import BaseEstimator, ClassifierMixin, clone
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

In [100]:
class SimpleMultiClassBoosting(BaseEstimator, ClassifierMixin):
    def __init__(self, base_estimator=None, n_estimators=50):
        self.base_estimator = base_estimator if base_estimator is not None else DecisionTreeClassifier(max_depth=1)
        self.n_estimators = n_estimators
        self.learners = []
        self.learner_weights = []
        self.label_encoder = LabelEncoder()

    def fit(self, X, y):
        # Convert labels to [0, n_classes-1]
        Y = self.label_encoder.fit_transform(y)
        n_classes = len(np.unique(Y))
        # Initialize weights uniformly
        n_samples = X.shape[0]
        self.weights = (1/n_samples) * np.ones(n_samples, dtype=float)
        for _ in range(self.n_estimators):
            learner = DecisionTreeClassifier(max_depth=1)
            learner.fit(X, Y, sample_weight=self.weights)
            pred = learner.predict(X)
            missClassified = pred != y
            learner_error = np.sum(self.weights[missClassified]) / np.sum(self.weights)
            # Compute weighted error rate (misclassification rate)
            learner_weight = np.log((1-learner_error) / learner_error) + np.log(n_classes - 1)
            # Compute learner weight using SAMME algorithm

            if learner_error >= 1 - (1 / n_classes):
                break

            # Increase the weights of misclassified samples
            for idx in range(n_samples):
                if missClassified[idx] == True:
                    self.weights[idx] *= np.exp(learner_weight)    
            self.weights /= np.sum(self.weights)    
            # Save the current learner
            self.learners.append(learner)
            self.learner_weights.append(learner_weight)

    def predict(self, X):
        # Collect predictions from each learner

        # Weighted vote for each sample's prediction across all learners

        # Final prediction is the one with the highest weighted vote

        # Convert back to original class labels
        predictionsOfLearners = []
        for learner in self.learners:
            predictionsOfLearners.append(learner.predict(X))
        predictionsOfLearners = np.array(predictionsOfLearners)
        prediction = np.empty(X.shape[0])        
        for i in range(X.shape[0]):
            labels = np.unique(predictionsOfLearners[:, i])
            votes = {label : 0 for label in labels}
            for j in range(len(predictionsOfLearners[:, i])):
                for label in labels:
                    if predictionsOfLearners[j, i] == label:
                        votes[label] += self.learner_weights[j]
            finalPrediction = max(votes, key=votes.get)
            prediction[i] = self.label_encoder.inverse_transform(np.array([finalPrediction]))
        return prediction


In [101]:
iris = load_iris()
df = pd.DataFrame(data=np.c_[iris['data'], iris['target']], columns=iris['feature_names'] + ['target'])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [102]:
X_train, X_test, Y_train, Y_test = train_test_split(df.iloc[:, :-1].to_numpy(), df.iloc[:, -1].to_numpy(), test_size=0.3, random_state=42)

In [103]:
m = SimpleMultiClassBoosting()
m.fit(X_train, Y_train)

In [104]:
prediction = m.predict(X_test)
prediction

array([1., 0., 2., 1., 1., 0., 1., 2., 1., 1., 2., 0., 0., 0., 0., 1., 2.,
       1., 1., 2., 0., 2., 0., 2., 2., 2., 2., 2., 0., 0., 0., 0., 1., 0.,
       0., 2., 1., 0., 0., 0., 2., 1., 1., 0., 0.])

In [105]:
print(m.learner_weights)

[1.3017369730918267, 2.114250946345696, 2.4361903096894206, 2.6001291918124525, 2.105163395441564, 1.6480070039057666, 2.014188780803626, 1.7398419334069946, 2.4093065069579587, 1.9564666410141864, 1.1076657447143285, 1.8142058424030298, 2.0023753103226927, 1.626581259630758, 1.6524999592072571, 1.819583010804596, 1.5679125249855816, 1.682873939578077, 1.8018571731946804, 1.4086633586172383, 1.9371166741691925, 1.7102962259830043, 1.131127046262888, 1.650648363172479, 1.600562850160947, 1.7366014772186413, 1.8453542583246687, 1.5451650715319487, 1.9992195889999356, 1.5533313672483562, 1.8302788893076096, 1.5716059543635796, 1.7916874972003147, 1.571902237661443, 1.472260463269306, 1.5767988870923795, 1.6304574666252014, 1.3247744008431108, 1.4931539072445987, 1.6995581922250098, 1.68096475031206, 1.4266263810163229, 1.7257834743329088, 1.498428479204587, 1.580295225195597, 1.7014079894377163, 1.6093806022458237, 1.6011434436341658, 1.6376034800293464, 1.6164216788635986]


In [106]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, prediction))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        19
         1.0       1.00      1.00      1.00        13
         2.0       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

